In [7]:
import os
import torch
import torchvision
from torch import nn

In [8]:

# !mkdir -p ~/.kaggle

# from google.colab import files
# uploaded = files.upload()

# !cp kaggle.json ~/.kaggle/

# !chmod 600 ~/.kaggle/kaggle.json

In [9]:
# !kaggle datasets download -d shubhamgoel27/dermnet

# !unzip dermnet.zip -d dermnet

In [10]:
# !ls dermnet

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# @title Default title text
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

# 定义预处理步骤
transform_train = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 定义你想要加载的类别
selected_folders = ['Acne and Rosacea Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Atopic Dermatitis Photos',
 'Bullous Disease Photos',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
  'Melanoma Skin Cancer Nevi and Moles',
 'Eczema Photos']  # 替换为你想加载的类别

# train_data_dir = '/content/dermnet/train'
# test_data_dir = '/content/dermnet/test'
train_data_dir = '/kaggle/input/dermnet/train'
test_data_dir = '/kaggle/input/dermnet/test'

# 加载完整的训练数据和测试数据
df_train_full = datasets.ImageFolder(root=train_data_dir, transform=transform_train)
df_test_full = datasets.ImageFolder(root=test_data_dir, transform=transform_test)

# 获取类别名与索引的映射
# Original: class_to_idx = {v: k for k, v in df_train_full.class_to_idx.items()}  # 反转字典
# Changed: Using the original class_to_idx where keys are class names and values are indices
class_to_idx = df_train_full.class_to_idx

# 找出选定类别对应的索引
# Original: selected_idx = [class_to_idx[folder] for folder in selected_folders]
# Changed: Accessing the index using the class name as the key
selected_idx = [class_to_idx[folder] for folder in selected_folders if folder in class_to_idx]

# 过滤出选定类别的数据索引
train_indices = [i for i, (_, label) in enumerate(df_train_full) if label in selected_idx]
test_indices = [i for i, (_, label) in enumerate(df_test_full) if label in selected_idx]

# 使用 Subset 只加载指定类别的数据
df_train_subset = Subset(df_train_full, train_indices)
df_test_subset = Subset(df_test_full, test_indices)

# DataLoader
batch_size = 64
train_data_loader = DataLoader(df_train_subset, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(df_test_subset, batch_size=batch_size, shuffle=False)

# 查看加载数据的类别
class_names = [folder for folder in df_train_full.classes if folder in selected_folders]
print(class_names)

In [ ]:
# @title Default title text
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# train_data_dir = '/content/dermnet/train'
# test_data_dir = '/content/dermnet/test'
train_data_dir = '/kaggle/input/dermnet/train'
test_data_dir = '/kaggle/input/dermnet/test'
df_train = datasets.ImageFolder(root=train_data_dir, transform=transform_train)
df_test = datasets.ImageFolder(root=test_data_dir,transform=transform_test)

# DataLoader
batch_size = 64
train_data_loader = DataLoader(df_train, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(df_test, batch_size=batch_size, shuffle=False)



# show a batch data
images, labels = next(iter(train_data_loader))
print(f'Image batch shape: {images.shape}')
print(f'Label batch shape: {labels.shape}')

class_names = df_train.classes
print(class_names)

In [ ]:
test_data_loader
train_data_loader

In [ ]:
# @title Default title text
!pip install torch torchvision
!pip install efficientnet-pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#需要改动换掉这个类，其余的正常运行就好
class CustomVGG(nn.Module): #VGG16
    def __init__(self, num_classes):  
        super(CustomVGG, self).__init__()  
        # load VGG16  
        self.base_model = models.vgg16(pretrained=True)  
          
        # Remove the last fully connected layer  
        self.base_model.classifier[6] = nn.Identity()  # The last layer in VGG's classifier is a fully connected layer  
          
        # Adapt the input size of the new fully connected layers  
        # VGG16 outputs 4096 features (after the adaptive average pooling and flattening) from its last convolutional + pooling layers  
        self.fc1 = nn.Linear(4096, 512)  
        self.fc2 = nn.Linear(512, num_classes)  
          
        self.relu = nn.ReLU()  
  
    def forward(self, x):  
        x = self.base_model(x)  
        x = x.view(x.size(0), -1)  # Flatten the tensor. This is usually done automatically by the fully connected layer, but we do it explicitly here for clarity.  
        x = self.relu(self.fc1(x))  
        x = self.fc2(x)  
        return x

num_classes = 23
model = CustomVGG(num_classes=num_classes).to(device)

In [ ]:
# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
import matplotlib.pyplot as plt
import torch.optim as optim

# 定义早停法参数  
early_stopping_patience = 10  
early_stopping_counter = 0  
best_val_loss = float('inf')  
stop_training = False  # 手动停止训练的标志  
  
# 定义ReduceLROnPlateau调度器  
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)  
  
num_epochs = 70  
train_losses = []  
train_accuracies = []  
val_losses = []  
val_accuracies = []  
  
fig, axs = plt.subplots(1, 2, figsize=(12, 5))  
loss_line1, = axs[0].plot([], [], label='Train Loss')  
loss_line2, = axs[0].plot([], [], label='Validation Loss')  
acc_line1, = axs[1].plot([], [], label='Train Accuracy')  
acc_line2, = axs[1].plot([], [], label='Validation Accuracy')  
  
def init():  
    for line in [loss_line1, loss_line2, acc_line1, acc_line2]:  
        line.set_data([], [])  
    return loss_line1, loss_line2, acc_line1, acc_line2  
  
def update(epoch):  
    axs[0].set_xlim(0, num_epochs)  
    axs[0].set_ylim(0, max(max(train_losses), max(val_losses)) * 1.1 if train_losses and val_losses else 1)  
    axs[1].set_ylim(0, 100)  
      
    loss_line1.set_data(range(1, epoch + 1), train_losses)  
    loss_line2.set_data(range(1, epoch + 1), val_losses)  
    acc_line1.set_data(range(1, epoch + 1), train_accuracies)  
    acc_line2.set_data(range(1, epoch + 1), val_accuracies)  
      
    return loss_line1, loss_line2, acc_line1, acc_line2  
  
ani = animation.FuncAnimation(fig, update, frames=num_epochs, init_func=init, blit=True, interval=1000)  # interval以毫秒为单位  
plt.show(block=False)  # 非阻塞模式显示动画  
  
for epoch in range(num_epochs): 
    # 训练代码（与原始代码相同，略去） 
    model.train()    
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_data_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (i + 1) % 5 == 0:
            batch_loss = running_loss / (i + 1)
            batch_accuracy = 100 * correct / total
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_data_loader)}], '
                  f'Train Loss: {batch_loss:.4f}, Train Accuracy: {batch_accuracy:.2f}%')
        
    avg_loss = running_loss / len(train_data_loader)
    avg_accuracy = 100 * correct / total
    train_losses.append(avg_loss)
    train_accuracies.append(avg_accuracy)
    print(f'Epoch [{epoch + 1}/{num_epochs}] completed. Average Train Loss: {avg_loss:.4f}, '
          f'Average Train Accuracy: {avg_accuracy:.2f}%')
    # 验证代码（与原始代码相同，略去）  
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in test_data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    avg_val_loss = val_running_loss / len(test_data_loader)
    avg_val_accuracy = 100 * val_correct / val_total
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_accuracy)

    print(f'Epoch [{epoch + 1}/{num_epochs}] completed. Validation Loss: {avg_val_loss:.4f}, '
          f'Validation Accuracy: {avg_val_accuracy:.2f}%')  
    # 更新学习率调度器  
    scheduler.step(avg_val_loss)  
      
    # 早停法逻辑  
    if avg_val_loss < best_val_loss:  
        best_val_loss = avg_val_loss  
        early_stopping_counter = 0  
    else:  
        early_stopping_counter += 1  
      
    # 检查是否应该停止训练  
    if early_stopping_counter >= early_stopping_patience or stop_training:  
        print(f'Early stopping at epoch {epoch + 1} due to no improvement in validation loss.')  
        break  
      
    # 更新动画（仅在本地运行时有效，Kaggle等在线环境不支持）  
    try:  
        ani.event_source.stop()  # 停止动画更新（如果之前已经开始的话）  
        ani.event_source = ani.new_timer(interval=1000)  # 重新设置定时器以继续更新（如果需要的话，但在这里我们其实不需要，因为下面会重新显示）  
        plt.close(fig)  # 关闭当前图形窗口（为了避免多个窗口同时打开）  
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # 重新创建图形和轴  
        # 重新初始化动画（略去，因为下面我们会保存并重新显示整个动画）  
        # ...  
        # 但实际上，在这里我们不需要重新初始化动画，因为我们会直接保存图像并退出循环  
    except:  
        pass  # 忽略在线环境不支持动画更新时的错误  
      
# 训练结束后保存曲线图（无论是因为完成所有epochs还是因为早停）  
plt.figure(figsize=(12, 5))  
plt.subplot(1, 2, 1)  
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')  
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')  
plt.xlabel('Epoch')  
plt.ylabel('Loss')  
plt.title('Training and Validation Loss')  
plt.legend()  
  
plt.subplot(1, 2, 2)  
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Train Accuracy')  
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')  
plt.xlabel('Epoch')  
plt.ylabel('Accuracy (%)')  
plt.title('Training and Validation Accuracy')  
plt.legend()  
  
plt.tight_layout()  
plt.savefig('training_curves.png')  # 保存曲线图  
plt.show()  # 在本地显示曲线图（Kaggle等在线环境可能不支持显示）

In [ ]:
# import matplotlib.pyplot as plt
# import torch.optim as optim

# # define ReduceLROnPlateau schedule
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# num_epochs = 70
# train_losses = []
# train_accuracies = []
# val_losses = []
# val_accuracies = []

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     for i, (images, labels) in enumerate(train_data_loader):
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)

#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#         if (i + 1) % 5 == 0:
#             batch_loss = running_loss / (i + 1)
#             batch_accuracy = 100 * correct / total
#             print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_data_loader)}], '
#                   f'Train Loss: {batch_loss:.4f}, Train Accuracy: {batch_accuracy:.2f}%')

#     avg_loss = running_loss / len(train_data_loader)
#     avg_accuracy = 100 * correct / total
#     train_losses.append(avg_loss)
#     train_accuracies.append(avg_accuracy)
#     print(f'Epoch [{epoch + 1}/{num_epochs}] completed. Average Train Loss: {avg_loss:.4f}, '
#           f'Average Train Accuracy: {avg_accuracy:.2f}%')

#     # validation
#     model.eval()
#     val_running_loss = 0.0
#     val_correct = 0
#     val_total = 0
#     with torch.no_grad():
#         for images, labels in test_data_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             loss = criterion(outputs, labels)

#             val_running_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             val_total += labels.size(0)
#             val_correct += (predicted == labels).sum().item()

#     avg_val_loss = val_running_loss / len(test_data_loader)
#     avg_val_accuracy = 100 * val_correct / val_total
#     val_losses.append(avg_val_loss)
#     val_accuracies.append(avg_val_accuracy)

#     print(f'Epoch [{epoch + 1}/{num_epochs}] completed. Validation Loss: {avg_val_loss:.4f}, '
#           f'Validation Accuracy: {avg_val_accuracy:.2f}%')

#     # ReduceLROnPlateau learning rate
#     scheduler.step(avg_val_loss)

# # loss diagram
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
# plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()

# # accuracy diagram
# plt.subplot(1, 2, 2)
# plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy')
# plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.title('Training and Validation Accuracy')
# plt.legend()

# plt.tight_layout()
# plt.show()


In [ ]:
# save
# torch.save(model.state_dict(), 'resnet50_skin_disease_model.pth')
torch.save(model, 'vgg16_skin_disease_model.pth')

In [ ]:

# from google.colab import files
# files.download('vgg16_skin_disease_model.pth')


In [ ]:

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')


from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# classifier report
print(classification_report(y_true, y_pred, target_names=class_names))

# confusion matrix
cm = confusion_matrix(y_true, y_pred)

# The confusion matrix heat map
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
